In [1]:
import pandas as pd

## Modelo - RandomForestRegressor

In [2]:
df = pd.read_csv('train_cleared.csv')

In [3]:
df

,fare_amount,passenger_count,distance,day_status,time_status
0,7.7,1,2.802346,0,0
1,7.5,1,1.557516,1,3
2,16.5,1,4.160010,1,2
3,9.0,1,1.254608,1,2
4,5.3,1,1.376087,0,1
...,...,...,...,...,...
8487132,5.5,3,0.872517,1,1
8487133,5.7,3,1.664409,1,1
8487134,13.7,1,2.528474,1,3
8487135,4.5,1,1.163118,0,3


In [3]:
selected_features = [
    'passenger_count',
    'distance',
    'day_status',
    'time_status'
]

X = df[selected_features]
y = df['fare_amount']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [7]:
std_scaler = StandardScaler()
std_scaler.fit(X * 1.0)

X_train_std = std_scaler.transform(X_train * 1.0)
X_test_std = std_scaler.transform(X_test * 1.0)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

aea = AdaBoostRegressor()

In [6]:
aea.fit(X_train, y_train)

AdaBoostRegressor()

In [7]:
aea.score(X_test, y_test)

0.6385213653481419